In [7]:
from pathlib import Path

import automech
from project_utilities import p_, util, workflow

file = util.notebook_file() if util.is_notebook() else __file__
tag = util.file_tag(file)
root_path = Path("..")
par_mech = workflow.read_parent_mechanism(root_path=root_path)

In [8]:
# Generate submechanism
from automol.graph import enum

gen_mech = automech.from_smiles(
    spc_smis=["C1=C[CH]CC1", "C1=CC[CH]C1"],
    src_mech=par_mech,
)
#  - H-migrations
gen_mech = automech.enumerate_reactions(
    gen_mech,
    enum.ReactionSmarts.h_migration_12,
    src_mech=par_mech,
)
#  - Ring-opening scissions
gen_mech = automech.enumerate_reactions(
    gen_mech,
    enum.ReactionSmarts.ring_beta_scission,
    src_mech=par_mech,
)
gen_mech = automech.enumerate_reactions(
    gen_mech,
    enum.ReactionSmarts.vinyl_ring_beta_scission,
    src_mech=par_mech,
)
#  - Beta scissions
gen_mech = automech.enumerate_reactions(
    gen_mech,
    enum.ReactionSmarts.beta_scission,
    src_mech=par_mech,
)
gen_mech = automech.enumerate_reactions(
    gen_mech,
    enum.ReactionSmarts.vinyl_beta_scission,
    src_mech=par_mech,
)
automech.display(gen_mech)

  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
# Expand stereochemistry
ste_mech, *_, gen_mech = workflow.expand_stereo(
    mech=gen_mech, tag=tag, root_path=root_path
)


Expanding stereochemistry...


  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]


Stereoexpansion errors:


In [10]:
# Prepare calculation
workflow.prepare_calculation(
    gen_mech=gen_mech, ste_mech=ste_mech, tag=tag, root_path=root_path
)


Sorting mechanism...

Writing mechanism...
../data/1e_C_r_v0_gen.json
../data/1e_C_r_v0_ste.json
../data/mechanalyzer/1e_C_r_v0_ste.dat
../data/mechanalyzer/1e_C_r_v0_ste.csv


  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]